In [8]:
import numpy as np
import random
import tensorflow as tf
from sympy import symbols, diff, lambdify
from tensorflow import keras
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras import layers
from keras.layers import *
from keras import regularizers
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [9]:
classes = 10
input_shape = (32, 32, 3)
n=5000
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train[1:n]
y_train = y_train[1:n]
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

y_train = to_categorical(y_train, classes)
y_test = to_categorical(y_test, classes)

In [10]:
def nn_model(alpha, beta1, beta2, init='uniform'):
    model = keras.Sequential(
        [
            keras.Input(shape=(28, 28, 1)),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(classes, activation="softmax"),
        ]
    )
    
    adam = tf.keras.optimizers.Adam(learning_rate=alpha, beta_1=beta1, beta_2=beta2)
    model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

    return model

In [11]:
model = nn_model(0.001, 0.9, 0.999)

In [12]:
def fit_nn(model):

  nmodel = KerasClassifier(build_fn=model, verbose=1)

  batch_size = [32, 64, 128, 512]
  epochs = [10, 20, 50, 100]

  params = dict(epochs=epochs, batch_size=batch_size)

  gsresult = GridSearchCV(estimator=nmodel, param_grid=params, cv=3)
  final_model = gsresult.fit(x_train, y_train)
  mts = final_model.cv_results_['mean_test_score']
  sts = final_model.cv_results_['std_test_score']
  params = final_model.cv_results_['params']
  for mean, st, param in zip(mts, sts, params):
      print(f'{mean:.4} || {st:.4} || {param}')

In [ ]:
fit_nn(model=model)